In [288]:
import os  # Manipulate Operating System
import win32gui, win32ui, win32con, win32clipboard  # Interface with Windows
import pandas as pd  # DataFrames
import numpy as np  # Array Transformation, Math
import warnings  # Ignore Warnings
import pyautogui  # Control Mouse and Keyboard
import re  # Regular Expressions for String Parsing

# Custom WindowCapture Class
from windowcapture import WindowCapture

# Warnings to Ignore
warnings.filterwarnings("ignore", category=UserWarning)

# Set to False so mouse cursor can be controlled beyond main screen
# Used due to multiple monitor setup
pyautogui.FAILSAFE = False

# Initialize WindowCapture class with name of NetPlay window
wincap = WindowCapture(window_name="NetPlay")
screenshot = wincap.get_screenshot()


In [289]:
screenshot.shape

(582, 723, 3)

In [290]:
player_location = pyautogui.locate(
    needleImage=r"C:\Users\Cooper\cv-mk64\img\netplay\Player.png",
    haystackImage=screenshot,
)

player_center = pyautogui.center(player_location)

mapping_location = pyautogui.locate(
    needleImage=r"C:\Users\Cooper\cv-mk64\img\netplay\Mapping.png",
    haystackImage=screenshot,
)

mapping_center = pyautogui.center(mapping_location)

print(f"Player Center: {player_center}")
print(f"Mapping Center: {mapping_center}")

player_screen_center = wincap.get_screen_position(player_center)
mapping_screen_center = wincap.get_screen_position(mapping_center)

print(f"Player Center Relative to Window Location on Screen: {player_screen_center}")
print(f"Mapping Center Relative to Window Location on Screen: {mapping_screen_center}")


Player Center: Point(x=383, y=133)
Mapping Center: Point(x=555, y=132)
Player Center Relative to Window Location on Screen: (-1379, 414)
Mapping Center Relative to Window Location on Screen: (-1207, 413)


In [291]:
player_screen_center[0]

-1379

In [292]:
def create_player_frame(
    player_location: tuple[int, int],
    mapping_location: tuple[int, int],
    wincap: WindowCapture,
) -> pd.DataFrame:

    # Translate Player Location Tuple into X and Y variables
    player_x = player_location[0]
    player_y = player_location[1]

    # Translate Mapping Location Tuple into X and Y variables
    mapping_x = mapping_location[0]
    mapping_y = mapping_location[1]

    # Bring the window to the front with the Window Handle ID
    win32gui.SetForegroundWindow(wincap.hwnd)

    # Abstraction for pyautogui interaction
    # Helper Function to copy selected cell value and paste into variable
    def click_helper(x: int, y: int, y_move: int) -> None:
        pyautogui.click(x, y)
        pyautogui.move(0, y_move)
        pyautogui.click()
        pyautogui.hotkey("ctrl", "c")
        
    def copy_paste() -> str:
        win32clipboard.OpenClipboard()
        data = win32clipboard.GetClipboardData()
        win32clipboard.EmptyClipboard()
        win32clipboard.CloseClipboard()
        
        return data

    # Copy Player Data from NetPlay Window
    click_helper(x=player_x, y=player_y, y_move=25)
    player_one = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=55)
    player_two = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=85)
    player_three = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=120)
    player_four = copy_paste()

    # Copy Controller Mapping (Port) Data from NetPlay Window
    click_helper(x=mapping_x, y=mapping_y, y_move=25)
    mapping_one = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=55)
    mapping_two = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=85)
    mapping_three = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=120)
    mapping_four = copy_paste()

    # Insert Values into Pandas DataFrame
    df = pd.DataFrame(
        data={
            "Player": [player_one, player_two, player_three, player_four],
            "Port": [mapping_one, mapping_two, mapping_three, mapping_four],
        }
    )

    # Remove rows with duplicated Player values
    df = df.drop_duplicates(subset="Player", keep="first")

    # Cast string to Capitalization (string -> String | STRING -> String)
    df["Player"] = df["Player"].str.capitalize()

    df["Port"]

    return df


In [293]:
df = create_player_frame(
    player_location=player_screen_center,
    mapping_location=mapping_screen_center,
    wincap=wincap,
)

df = df.replace("None", np.nan)

In [294]:
df

,Player,Port
0,Cooper,"GC1,2"
1,Blake,GC3
2,Connor,GC4
3,Cole,NaN


In [336]:
# Read Testing DataFrame

df = pd.read_csv(r"C:\Users\Cooper\cv-mk64\img\netplay\netplay_test.csv").drop(
    columns="Unnamed: 0"
)

df


,Player,Port
0,Cooper,"GC1,2"
1,Blake,GC3
2,Connor,GC4
3,Cole,NaN


In [337]:
pd.get_dummies(df)

,Player_Blake,Player_Cole,Player_Connor,Player_Cooper,"Port_GC1,2",Port_GC3,Port_GC4
0,0,0,0,1,1,0,0
1,1,0,0,0,0,1,0
2,0,0,1,0,0,0,1
3,0,1,0,0,0,0,0


In [324]:
for i in df["Port"]:
    print(i, type(i))
    

GC1,2 <class 'str'>
GC3 <class 'str'>
GC4 <class 'str'>
nan <class 'float'>


In [325]:
s = "1,2"

s.split(",")

['1', '2']

In [326]:
for i in df["Port"]:
    if isinstance(i, str):
        sub = re.sub(pattern="[a-zA-Z]", repl="", string=i)
        if "," in sub:
            for j in sub.split(","):
                df[f"Port_A{j}"] = j
        else:
            df["Port_A"] = sub
    else:
        df["Port_A"] = np.nan


In [328]:
tdf = pd.DataFrame({"one": [1, 2, 3, 4], "two": [1, 2, 3, 4]})


In [331]:
pd.crosstab(tdf["one"], tdf["two"]).reset_index()

two,one,1,2,3,4
0,1,1,0,0,0
1,2,0,1,0,0
2,3,0,0,1,0
3,4,0,0,0,1


In [335]:
pd.get_dummies(df)

,Port_A,Player_Blake,Player_Cole,Player_Connor,Player_Cooper,"Port_GC1,2",Port_GC3,Port_GC4,Port_A1_1,Port_A2_2
0,NaN,0,0,0,1,1,0,0,1,1
1,NaN,1,0,0,0,0,1,0,1,1
2,NaN,0,0,1,0,0,0,1,1,1
3,NaN,0,1,0,0,0,0,0,1,1


In [334]:
tdf

,one,two
0,1,1
1,2,2
2,3,3
3,4,4


In [327]:
df 

,Player,Port,Port_A1,Port_A2,Port_A
0,Cooper,"GC1,2",1,2,NaN
1,Blake,GC3,1,2,NaN
2,Connor,GC4,1,2,NaN
3,Cole,NaN,1,2,NaN


In [312]:
[
    re.sub(pattern="[a-zA-Z]", repl="", string=x).split(",")
    for x in df["Port"]
    if isinstance(x, str)
]


[['1', '2'], ['3'], ['4']]

In [275]:
pd.DataFrame({"one": ['1','2']})

,one
0,1
1,2


In [251]:
df["Port_split"] = df["Port"].str.replace(pat="[a-zA-Z]", repl="", regex=True).str.split(",")

df

ValueError: Length of values (2) does not match length of index (4)

In [265]:
# port_arr = df["Port"].str.replace(pat="[a-zA-Z]", repl="", regex=True).str.split(",")[0]

for idx, i in enumerate(df["Port"]):
    for j in i:
        s = re.sub("[a-zA-Z]", repl="", string=j).split(",")[idx]
        df[f"Port{i}"] = s

IndexError: list index out of range

In [266]:
df

,Player,Port,"PortGC1,2"
0,Cooper,"GC1,2",2
1,Blake,GC3,2
2,Connor,GC4,2
3,Cole,NaN,2


In [216]:
ones = np.ones(4)

for i in ones:
    print(i)

1.0
1.0
1.0
1.0


In [193]:
df

,Player,Port,"Port['1', '2']"
0,Cooper,"GC1,2",1


In [199]:
a = "GC1,2"

In [203]:
import re

In [202]:
a.replace(pat="[a-zA-Z]", repl="", regex=True)

TypeError: str.replace() takes no keyword arguments

In [205]:
re.sub(pattern="[a-zA-Z]", repl="", string=a).split(",")

['1', '2']